In [ ]:
import pandas as pd
import json
import numpy as np
from datetime import datetime

print("="*80)
print("FASE 2: QUALIDADE E CAMADA TRUSTED")
print("="*80)

# ===== LOAD RAW DATA =====
with open('../data/raw/products_raw.json', 'r') as f:
    products_raw = pd.DataFrame(json.load(f))

customers_raw = pd.read_csv('../data/raw/customers_raw.csv')
sales_raw = pd.read_csv('../data/raw/sales_raw.csv')

print(f"\n✓ Produtos carregados: {len(products_raw)}")
print(f"✓ Clientes carregados: {len(customers_raw)}")
print(f"✓ Vendas carregadas: {len(sales_raw)}")

# ===== TRANSFORMAÇÃO: PRODUCTS =====
print("\n" + "="*80)
print("TRANSFORMAÇÃO: PRODUCTS_RAW → PRODUCTS_TRUSTED")
print("="*80)

products_trusted = products_raw.copy()

# Tipo de dados
products_trusted['load_capacity'] = pd.to_numeric(products_trusted['load_capacity'], errors='coerce')
products_trusted['max_speed'] = pd.to_numeric(products_trusted['max_speed'], errors='coerce')
products_trusted['temperature_limit'] = pd.to_numeric(products_trusted['temperature_limit'], errors='coerce')
products_trusted['unit_cost'] = pd.to_numeric(products_trusted['unit_cost'], errors='coerce')
products_trusted['list_price'] = pd.to_numeric(products_trusted['list_price'], errors='coerce')

print("\n1️⃣  Tipos de dados padronizados")

# FIX CRÍTICO: Margem de lucro negativa
problematic_count = (products_trusted['unit_cost'] >= products_trusted['list_price']).sum()
print(f"\n2️⃣  Detectados {problematic_count} produtos com margem negativa")

# Corrigir: set list_price = unit_cost * 1.25 (margem mínima 25%)
products_trusted.loc[
    products_trusted['unit_cost'] >= products_trusted['list_price'],
    'list_price'
] = products_trusted.loc[
    products_trusted['unit_cost'] >= products_trusted['list_price'],
    'unit_cost'
] * 1.25

print(f"   ✓ Corrigidos com margem mínima de 25%")


# Adicionar campo de descrição técnica
def generate_technical_description(row):
    return f"""{row['product_name']} é um rolamento do tipo {row['bearing_type']} em {row['material']}, fabricado pela {row['manufacturer']} (modelo {row['model']}). 
    Capacidade de carga: {row['load_capacity']:.0f}N, velocidade máxima: {row['max_speed']:,.0f} RPM, 
    limite de temperatura: {row['temperature_limit']}°C. 
    Indicado para resolver problemas de {row['problem_type']}. 
    Preço: R$ {row['list_price']:.2f}, custo: R$ {row['unit_cost']:.2f}."""

products_trusted['technical_description'] = products_trusted.apply(
    generate_technical_description, axis=1
)

print("3️⃣  Campo technical_description adicionado")

# Validar
assert (products_trusted['unit_cost'] < products_trusted['list_price']).all(), \
    "Ainda existem produtos com unit_cost >= list_price!"

assert products_trusted.isnull().sum().sum() == 0, \
    "Existem valores nulos!"

print("4️⃣  Validações passaram ✓")

# ===== TRANSFORMAÇÃO: CUSTOMERS =====
print("\n" + "="*80)
print("TRANSFORMAÇÃO: CUSTOMERS_RAW → CUSTOMERS_TRUSTED")
print("="*80)

customers_trusted = customers_raw.copy()

# Parse data columns
customers_trusted['relationship_start_date'] = pd.to_datetime(customers_trusted['relationship_start_date'])
customers_trusted['last_updated'] = pd.to_datetime(customers_trusted['last_updated'])

# Numeric types
customers_trusted['annual_revenue_estimated'] = pd.to_numeric(
    customers_trusted['annual_revenue_estimated'], errors='coerce'
)
customers_trusted['maintenance_budget_annual'] = pd.to_numeric(
    customers_trusted['maintenance_budget_annual'], errors='coerce'
)
customers_trusted['downtime_cost_per_hour'] = pd.to_numeric(
    customers_trusted['downtime_cost_per_hour'], errors='coerce'
)

print("1️⃣  Tipos de dados padronizados")

# Mapear problemas esperados por indústria
industry_problems = {
    'Mineração': ['Vibração', 'Desgaste'],
    'Siderurgia': ['Superaquecimento', 'Vibração'],
    'Alimentos': ['Contaminação', 'Desgaste'],
    'Automotiva': ['Vibração', 'Superaquecimento'],
    'Papel e Celulose': ['Desgaste', 'Contaminação'],
    'Química': ['Contaminação', 'Superaquecimento'],
    'Cimento': ['Desgaste', 'Vibração'],
    'Energia': ['Superaquecimento', 'Vibração']
}

customers_trusted['expected_problem_type'] = customers_trusted['industry'].map(industry_problems)

print("2️⃣  Mapeamento de problemas esperados por indústria")

assert customers_trusted.isnull().sum().sum() == 0, "Existem valores nulos!"

print("3️⃣  Validações passaram ✓")

# ===== TRANSFORMAÇÃO: SALES =====
print("\n" + "="*80)
print("TRANSFORMAÇÃO: SALES_RAW → SALES_TRUSTED")
print("="*80)

sales_trusted = sales_raw.copy()

# Parse dates
sales_trusted['sale_date'] = pd.to_datetime(sales_trusted['sale_date'])
sales_trusted['last_updated'] = pd.to_datetime(sales_trusted['last_updated'])

# Numeric types
sales_trusted['quantity'] = pd.to_numeric(sales_trusted['quantity'], errors='coerce').astype('int64')
sales_trusted['unit_price'] = pd.to_numeric(sales_trusted['unit_price'], errors='coerce')
sales_trusted['total_price'] = pd.to_numeric(sales_trusted['total_price'], errors='coerce')

print("1️⃣  Tipos de dados padronizados")

# Validar cálculo de total_price
tolerance = 0.01
mismatches = (
    abs(sales_trusted['total_price'] - (sales_trusted['quantity'] * sales_trusted['unit_price'])) 
    > tolerance
).sum()

if mismatches > 0:
    print(f"⚠️  {mismatches} registros com total_price inconsistente")
    # Corrigir
    sales_trusted['total_price'] = sales_trusted['quantity'] * sales_trusted['unit_price']
    print(f"   ✓ Corrigidos")
else:
    print("2️⃣  Total_price validado (100% correto)")

assert sales_trusted.isnull().sum().sum() == 0, "Existem valores nulos!"

print("3️⃣  Validações passaram ✓")

# ===== SALVAR TRUSTED LAYER =====
print("\n" + "="*80)
print("SALVANDO TRUSTED LAYER")
print("="*80)

import os
os.makedirs('../data/trusted', exist_ok=True)

products_trusted.to_parquet('../data/trusted/products_trusted.parquet', index=False)
customers_trusted.to_parquet('../data/trusted/customers_trusted.parquet', index=False)
sales_trusted.to_parquet('../data/trusted/sales_trusted.parquet', index=False)

print("✓ products_trusted.parquet")
print("✓ customers_trusted.parquet")
print("✓ sales_trusted.parquet")

# ===== RELATÓRIO FINAL =====
print("\n" + "="*80)
print("RELATÓRIO FINAL - PHASE 2")
print("="*80)

print(f"""
RESUMO DAS TRANSFORMAÇÕES:

Products:
  • Registros: {len(products_trusted):,}
  • Campos: {len(products_trusted.columns)}
  • Nulos: {products_trusted.isnull().sum().sum()}
  • Margens corrigidas: {problematic_count}
  • Descrições técnicas: Adicionadas ✓

Customers:
  • Registros: {len(customers_trusted):,}
  • Campos: {len(customers_trusted.columns)}
  • Nulos: {customers_trusted.isnull().sum().sum()}
  • Problemas por indústria: Mapeados ✓

Sales:
  • Registros: {len(sales_trusted):,}
  • Campos: {len(sales_trusted.columns)}
  • Nulos: {sales_trusted.isnull().sum().sum()}
  • Total price validado: {(sales_trusted['quantity'] * sales_trusted['unit_price']).round(2).equals(sales_trusted['total_price'].round(2))}

✅ PHASE 2 CONCLUÍDA COM SUCESSO!

Próximo: PHASE 3 - Modelagem de Dados (Camada Refined)
""")
